In [0]:
#path=r'/Volumes/databricks_lamba/bronze/bronzevol/Sales/fact_sales.csv'
path=r'/Volumes/databricks_lamba/bronze/bronzevol/customer/dim_customers.csv'

In [0]:
df= spark.read.csv(path,header=True,inferSchema=True)
display(df)

In [0]:
df= spark.read.format('csv')\
    .option("header",True)\
    .option("inferSchema",True)\
    .load('/Volumes/databricks_lamba/bronze/bronzevol/Sales/fact_sales.csv')

In [0]:
df.show()

In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df= df.withColumn('name',upper(col('name')))
df.show()

In [0]:
df= df.withColumn('domain',split(col('email'),'@')[1])
df.display()

In [0]:
from pyspark.sql.functions import *
display(df.groupBy('domain').agg(count('customer_id').alias('customer_count')).sort(col('customer_count').desc()))

####
 **UPSERT**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_lamba.silver.customer_env"):

   dlt_obj = DeltaTable.forName(spark,'databricks_lamba.silver.customer_env')

   dlt_obj.alias("trg").merge(df.alias("src"),"trg.customer_id=src.customer_id")\
   .whenMatchedUpdateAll()\
   .whenNotMatchedInsertAll()\
   .execute()

else:
    df.write.format("delta")\
    .mode("append")\
    .saveAsTable("databricks_lamba.silver.customer_env")

In [0]:
%sql
select *from databricks_lamba.silver.customer_env

In [0]:
df_prod= spark.read.format('csv')\
    .option("header",True)\
    .option("inferSchema",True)\
    .load('/Volumes/databricks_lamba/bronze/bronzevol/product/dim_products.csv')
df_prod.display()

In [0]:
from pyspark.sql.functions import *
df_prod = df_prod.withColumn('process_date',current_timestamp())
df_prod.display()

In [0]:
display(df_prod.groupBy('category').agg(avg('price').alias('avg_price')))


In [0]:
if spark.catalog.tableExists("databricks_lamba.silver.product_enr"):

   dlt_obj = DeltaTable.forName(spark,'databricks_lamba.silver.product_enr')

   dlt_obj.alias("trg").merge(df_prod.alias("src"),"trg.product_id=src.product_id")\
   .whenMatchedUpdateAll()\
   .whenNotMatchedInsertAll()\
   .execute()

else:
    df_prod.write.format("delta")\
    .mode("append")\
    .saveAsTable("databricks_lamba.silver.product_enr")

In [0]:
%sql
select * from databricks_lamba.silver.product_enr

In [0]:
df_str= spark.read.format('csv')\
    .option("header",True)\
    .option("inferSchema",True)\
    .load('/Volumes/databricks_lamba/bronze/bronzevol/stores/dim_stores.csv')
df_str.display()

In [0]:
from pyspark.sql.functions import *
df_str = df_str.withColumn('store_name',regexp_replace(col('store_name'),"_",""))
df_str.display()

In [0]:
df_str = df_str.withColumn('process_date',current_timestamp())
df_str.display()


In [0]:
if spark.catalog.tableExists("databricks_lamba.silver.store_enr"):

   dlt_obj = DeltaTable.forName(spark,'databricks_lamba.silver.store_enr')

   dlt_obj.alias("trg").merge(df_str.alias("src"),"trg.store_id=src.store_id")\
   .whenMatchedUpdateAll()\
   .whenNotMatchedInsertAll()\
   .execute()

else:
    df_str.write.format("delta")\
    .mode("append")\
    .saveAsTable("databricks_lamba.silver.store_enr")

In [0]:
%sql
select *from databricks_lamba.silver.store_enr

In [0]:
df_sales= spark.read.format('csv')\
    .option("header",True)\
    .option("inferSchema",True)\
    .load('/Volumes/databricks_lamba/bronze/bronzevol/Sales/fact_sales.csv')
df_sales.display()

In [0]:
df_sales = df_sales.withColumn('pricePerSale',round(col('total_amount')/col('quantity'),2))
df_sales = df_sales.withColumn('process_date',current_timestamp())
df_sales.display()

In [0]:
if spark.catalog.tableExists("databricks_lamba.silver.sales_enr"):

   dlt_obj = DeltaTable.forName(spark,'databricks_lamba.silver.sales_enr')

   dlt_obj.alias("trg").merge(df_sales.alias("src"),"trg.sales_id=src.sales_id")\
   .whenMatchedUpdateAll()\
   .whenNotMatchedInsertAll()\
   .execute()

else:
    df_sales.write.format("delta")\
    .mode("append")\
    .saveAsTable("databricks_lamba.silver.sales_enr")

In [0]:
%sql
select * from databricks_lamba.silver.sales_enr

#### **SPARK SQL**

In [0]:
display(spark.sql("select * from databricks_lamba.silver.sales_enr"))

In [0]:
#df= spark.sql("select UPPER(category) from databricks_lamba.silver.product_enr")

df= spark.sql("select * from databricks_lamba.silver.product_enr")

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView('Product_view')

In [0]:
df= spark.sql ("""
           Select *,
                   CASE
                   when category ='Toys' then 'Yes' else 'No' END AS flag
                   from Product_view
           """)

In [0]:
df.display()

#### **Pyspark UDF**

In [0]:
def greet(p_input):
    return "Hello"+str(p_input)

In [0]:
from pyspark.sql.functions import *
udf_greet = udf(greet)
df=df.withColumn('greet',udf_greet(col('flag')))
df.display()